<a href="https://colab.research.google.com/github/YinYinTan/CIND820FinalProject/blob/main/Data_Transformation_%26_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip3 install pandas_profiling --upgrade

     |████████████████████████████████| 261 kB 5.2 MB/s 
     |████████████████████████████████| 303 kB 35.5 MB/s 
     |████████████████████████████████| 10.9 MB 56.0 MB/s 
     |████████████████████████████████| 675 kB 53.0 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
     |████████████████████████████████| 3.1 MB 39.6 MB/s 
     |████████████████████████████████| 596 kB 43.0 MB/s 
     |████████████████████████████████| 102 kB 9.7 MB/s 
     |████████████████████████████████| 812 kB 53.6 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=eaf60f8b69fa05b57ad7ae41b9bc83373305ac4a1352e4cfd56f1da39d8d8663
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295206 sha256=c846046361d5cdc120bd5d639c293509584f3601db07615593d16c741d0c6c22
  

# Data Transformation

In [1]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

#Read file
df = pd.read_csv('HS.csv', index_col=None)

#Drop rows containing observations that are not valid for the project
df.drop(df[df['EMP_30'] != 1].index, inplace = True)
df.drop(df[df['GEN_10'] == 9].index, inplace = True)
df.drop(df[df['GEN_15'] == 9].index, inplace = True)

#Create new derived variable, which will be the target variable for this project
df['Worse_MH'] = np.where((df['GEN_10'] >= 4) & (df['GEN_15'] >= 4), 0, 1)

#Drop columns that are not needed for the project
df = df.drop(columns=['PUMFID', 'VERDATE', 'EMP_05', 'BMF_P', 'EMP_30', 'GEN_10', 'GEN_15'])

#Drop columns pertaining to derived variables that are not needed for the project
df = df.drop(columns=['PPEDVEY1','PPEDVEY2','PPEDVFV1','PPEDVFV2','PPEDVGL1','PPEDVGL2','PPEDVGN1','PPEDVGN2','PPEDVHS1','PPEDVHS2','PPEDVMA1','PPEDVMA2','PPEDVOT1','PPEDVOT2','PPEDVRE1','PPEDVRE2','PPEDVRS1','PPEDVRS2','GENDVHDI','GENDVMHI'])

#If PPE_10 = “No” = 2, then PPE_15A, PPE_15B, PPE_15C, PPE_15D, PPE_15E, PPE_15F, PPE_15G, PPE_15H, PPE_15I, and PPE_15J = “No” = 2 (instead of "Valid Skip")
skip_set_1 = ['PPE_15A','PPE_15B','PPE_15C','PPE_15D','PPE_15E','PPE_15F','PPE_15G','PPE_15H','PPE_15I','PPE_15J']
df[skip_set_1] = np.where(df[['PPE_10']] == 2, 2, df[skip_set_1])

#If PPE_20 = “No” = 2, then PPE_30A, PPE_30B, PPE_30C, PPE_30D, PPE_30E, PPE_30F, PPE_30G, PPE_30H,...
#PPE_35A, PPE_35B, PPE_35C, PPE_35D, PPE_35E, PPE_35F, PPE_35G, PPE_35H = "Not needed for job" = 1 (instead of "Valid Skip")
skip_set_2 = ['PPE_30A','PPE_30B','PPE_30C','PPE_30D','PPE_30E','PPE_30F','PPE_30G','PPE_30H','PPE_35A','PPE_35B','PPE_35C','PPE_35D','PPE_35E','PPE_35F','PPE_35G','PPE_35H']
df[skip_set_2] = np.where(df[['PPE_20']] == 2, 1, df[skip_set_2])

#If PPE_20 = “No” = 2, then PPE_25, PPE_40A, PPE_40B, PPE_40C, PPE_40D, PPE_40E, PPE_40F, PPE_40G, PPE_40H, PPE_40I,...
#PPE_45A, PPE_45B, PPE_45C, PPE_45D, PPE_45E, PPE_45F, PPE_45G, PPE_45H, PPE_45I = "No" = 2 (instead of "Valid Skip")
skip_set_3 = ['PPE_25','PPE_40A','PPE_40B','PPE_40C','PPE_40D','PPE_40E','PPE_40F','PPE_40G','PPE_40H','PPE_40I','PPE_45A','PPE_45B','PPE_45C','PPE_45D','PPE_45E','PPE_45F','PPE_45G','PPE_45H','PPE_45I']
df[skip_set_3] = np.where(df[['PPE_20']] == 2, 2, df[skip_set_3])

#If response to PPE access question = “Not stated” = 9 (i.e. missing values), then update to "Not needed for job" = 1
missing_set_1 = ['PPE_30A','PPE_30B','PPE_30C','PPE_30D','PPE_30E','PPE_30F','PPE_30G','PPE_30H','PPE_35A','PPE_35B','PPE_35C','PPE_35D','PPE_35E','PPE_35F','PPE_35G','PPE_35H']
for col in missing_set_1:
  df[col].mask(df[col] == 9, 1, inplace=True)

#If response to PPE restriction question = “Not stated” = 9 (i.e. missing values), then update to "No" = 2
missing_set_2 = ['PPE_40A','PPE_40B','PPE_40C','PPE_40D','PPE_40E','PPE_40F','PPE_40G','PPE_40H','PPE_40I','PPE_45A','PPE_45B','PPE_45C','PPE_45D','PPE_45E','PPE_45F','PPE_45G','PPE_45H','PPE_45I']
for col in missing_set_2:
  df[col].mask(df[col] == 9, 2, inplace=True)

#If response to PPE or IPC protocol/practice question = “Not stated” = 9 (i.e. missing values), then remove from dataset
df.drop(df[(df['PPE_05'] == 9)|(df['PPE_10'] == 9)|(df['PPE_15A'] == 9)|(df['PPE_15B'] == 9)|(df['PPE_15C'] == 9)|(df['PPE_15D'] == 9)|(df['PPE_15E'] == 9)|(df['PPE_15F'] == 9)|(df['PPE_15G'] == 9)|(df['PPE_15H'] == 9)|(df['PPE_15I'] == 9)|(df['PPE_15J'] == 9)].index, inplace = True)
df.drop(df[(df['PPE_20'] == 9)|(df['PPE_25'] == 9)].index, inplace = True)
df.drop(df[(df['PPE_50A'] >= 96)|(df['PPE_50B'] >= 96)|(df['PPE_50C'] >= 96)|(df['PPE_50D'] >= 96)|(df['PPE_50E'] >= 96)|(df['PPE_50F'] >= 96)].index, inplace = True)

#Handling of 14/16 remaining variables with classes of "Not stated" = 9 (i.e. missing values): Reassign to most frequent value
missing_set_3 = ['EMP_10','EMP_35','EMP_45','EMPDVGOC','ENV_30','ENVDVCON','ENVDVTYP','ENVDVGRW','GEN_05','GEN_20','AGEDVG4','GDRDVGRP','ISDVFLAG','PGDVFLA','IMMDVGST']
for col in missing_set_3:
  frequent = df[col].mode()
  df[col].mask(df[col] == 9, frequent, inplace=True)

#One-hot key encoding
'''
In progress
'''

#Change all variables to categorical or else they will be considered numeric
df = df.astype('category')

#View basic summary of variables
df.describe()
#df.head()

#df.to_csv('HS - revised.csv')

,GEODVGPR,EMP_10,EMP_35,EMP_45,EMPDVGOC,EMPDVGYW,ENV_25A,ENV_25B,ENV_25C,ENV_30,ENVDVCON,ENVDVTYP,ENVDVGRW,PPE_05,PPE_10,PPE_15A,PPE_15B,PPE_15C,PPE_15D,PPE_15E,PPE_15F,PPE_15G,PPE_15H,PPE_15I,PPE_15J,PPE_20,PPE_25,PPE_30A,PPE_30B,PPE_30C,PPE_30D,PPE_30E,PPE_30F,PPE_30G,PPE_30H,PPE_35A,PPE_35B,PPE_35C,PPE_35D,PPE_35E,PPE_35F,PPE_35G,PPE_35H,PPE_40A,PPE_40B,PPE_40C,PPE_40D,PPE_40E,PPE_40F,PPE_40G,PPE_40H,PPE_40I,PPE_45A,PPE_45B,PPE_45C,PPE_45D,PPE_45E,PPE_45F,PPE_45G,PPE_45H,PPE_45I,PPE_50A,PPE_50B,PPE_50C,PPE_50D,PPE_50E,PPE_50F,GEN_05,GEN_20,AGEDVG4,GDRDVGRP,ISDVFLAG,PGDVFLA,IMMDVGST,Worse_MH
count,17319,16451.0,16777.0,17296.0,16271.0,17319,17319,17319,17319,16638.0,16655.0,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17319,17307.0,17260.0,17284.0,17249.0,17005.0,16752.0,16907.0,17319
unique,7,2.0,2.0,3.0,9.0,4,3,3,3,3.0,3.0,7,8,3,2,2,2,2,2,2,2,2,2,2,2,2,2,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,6,6,6,6,6,6,5.0,5.0,4.0,2.0,2.0,2.0,2.0,2
top,30,2.0,1.0,1.0,5.0,1,2,1,2,2.0,2.0,1,30,1,1,1,1,1,1,1,1,1,1,2,1,1,1,2,2,1,2,2,2,2,1,2,2,1,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,4,2,2.0,4.0,1.0,2.0,2.0,2.0,1.0,1
freq,7898,13268.0,12341.0,12613.0,6443.0,5661,10207,11599,15596,10482.0,9148.0,7259,7879,14572,16534,15527,15123,13247,15136,13074,13836,10252,12874,14392,9437,16810,13718,13394,14131,7234,9984,9790,9540,13783,13166,13787,15001,7273,10769,10757,10237,14354,12555,9617,10148,14165,12502,16990,14910,15823,15405,12555,12438,12165,15059,14274,17135,15974,16541,16353,10131,7423,7229,6296,7189,4061,5335,6905.0,7245.0,5012.0,15246.0,16751.0,15036.0,15064.0,11929


# Exploratory Data Analysis (EDA)

In [2]:
profile = ProfileReport(df)
profile.to_file(output_file="Dataset Profile.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/pivot.py:188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agged[v] = maybe_downcast_to_dtype(agged[v], data[v].dtype)


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]